In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [6]:
color = {
    "U": 0,
    "D": 1,
    "F": 2,
    "R": 3,
    "B": 4,
    "L": 5
}

In [ ]:
class ResnetModel(nn.Module):
    def __init__(self, state_dim: int, one_hot_depth: int, h1_dim: int, resnet_dim: int, num_resnet_blocks: int,
                 out_dim: int, batch_norm: bool):
        super().__init__()
        self.one_hot_depth: int = one_hot_depth
        self.state_dim: int = state_dim
        self.blocks = nn.ModuleList()
        self.num_resnet_blocks: int = num_resnet_blocks
        self.batch_norm = batch_norm

        # first two hidden layers
        if one_hot_depth > 0:
            self.fc1 = nn.Linear(self.state_dim * self.one_hot_depth, h1_dim)
        else:
            self.fc1 = nn.Linear(self.state_dim, h1_dim)

        if self.batch_norm:
            self.bn1 = nn.BatchNorm1d(h1_dim)

        self.fc2 = nn.Linear(h1_dim, resnet_dim)

        if self.batch_norm:
            self.bn2 = nn.BatchNorm1d(resnet_dim)

        # resnet blocks
        for block_num in range(self.num_resnet_blocks):
            if self.batch_norm:
                res_fc1 = nn.Linear(resnet_dim, resnet_dim)
                res_bn1 = nn.BatchNorm1d(resnet_dim)
                res_fc2 = nn.Linear(resnet_dim, resnet_dim)
                res_bn2 = nn.BatchNorm1d(resnet_dim)
                self.blocks.append(nn.ModuleList([res_fc1, res_bn1, res_fc2, res_bn2]))
            else:
                res_fc1 = nn.Linear(resnet_dim, resnet_dim)
                res_fc2 = nn.Linear(resnet_dim, resnet_dim)
                self.blocks.append(nn.ModuleList([res_fc1, res_fc2]))

        # output
        self.fc_out = nn.Linear(resnet_dim, out_dim)

    def forward(self, states_nnet):
        x = states_nnet

        # preprocess input
        if self.one_hot_depth > 0:
            x = F.one_hot(x.long(), self.one_hot_depth)
            x = x.float()
            x = x.view(-1, self.state_dim * self.one_hot_depth)
        else:
            x = x.float()

        # first two hidden layers
        x = self.fc1(x)
        if self.batch_norm:
            x = self.bn1(x)

        x = F.relu(x)
        x = self.fc2(x)
        if self.batch_norm:
            x = self.bn2(x)

        x = F.relu(x)

        # resnet blocks
        for block_num in range(self.num_resnet_blocks):
            res_inp = x
            if self.batch_norm:
                x = self.blocks[block_num][0](x)
                x = self.blocks[block_num][1](x)
                x = F.relu(x)
                x = self.blocks[block_num][2](x)
                x = self.blocks[block_num][3](x)
            else:
                x = self.blocks[block_num][0](x)
                x = F.relu(x)
                x = self.blocks[block_num][1](x)

            x = F.relu(x + res_inp)

        # output
        x = self.fc_out(x)
        return x


In [61]:
state = "UUUUUUUUURRRRRRRRRFFFFFFFFFDDDDDDDDDLLLLLLLLLBBBBBBBBB"
state_for_tensor = []
for s in state:
    state_for_tensor.append(color[s])

tensor = torch.tensor(state_for_tensor)
tensor.shape

torch.Size([54])

In [60]:
net = ResnetModel(54, 6, 5000, 1000, 4, 1, False)

tensor = tensor.squeeze(0)
print(tensor.shape)

print(net(tensor))
print(net(tensor).item())

torch.Size([54])
tensor([[-0.0180]], grad_fn=<AddmmBackward0>)
-0.017974071204662323


In [40]:
import magiccube
from environment.algorithm import CFOP

"URFDLB"

state = "ORGBWGWOY" \
"        WWBOORYBB" \
"        OGRBGRYWB" \
"        GYRYRWRGB" \
"        WWGGBBOOG" \
"        ROWYYROYY"

c = magiccube.Cube(state=state)
print(c)
c.get_kociemba_facelet_positions()
r = CFOP().cross(c)
print(r)

          O  R  G                   
          B  W  G                   
          W  O  Y                   
 W  W  B  O  G  R  G  Y  R  W  W  G 
 O  O  R  B  G  R  Y  R  W  G  B  B 
 Y  B  B  Y  W  B  R  G  B  O  O  G 
          R  O  W                   
          Y  Y  R                   
          O  Y  Y                   

0.0
